In [4]:
!pip install tree_sitter tree_sitter_javascript tree_sitter_typescript tree_sitter_python


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
import json
import ast
from pathlib import Path
from tree_sitter import Language, Parser
import tree_sitter_javascript
import tree_sitter_typescript
import tree_sitter_python
from collections import defaultdict

SKIP_DIRECTORIES = {
    'node_modules', '.git', '__pycache__', 'dist', 'build', '.next',
    'coverage', '.pytest_cache', 'venv', 'env', '.venv', 'out'
}

LANGUAGE_EXTENSIONS = {
    '.js': 'javascript', '.mjs': 'javascript', '.jsx': 'jsx',
    '.ts': 'typescript', '.tsx': 'tsx', '.py': 'python'
}

def should_skip_directory(dirname):
    return dirname in SKIP_DIRECTORIES or dirname.startswith('.')

def get_file_language(filename):
    for ext, lang in LANGUAGE_EXTENSIONS.items():
        if filename.endswith(ext):
            return lang
    return None

class JavaScriptParser:
    def __init__(self):
        self.language = Language(tree_sitter_javascript.language())
        self.parser = Parser(self.language)
    
    def parse(self, file_path, rel_path, is_jsx=False):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except:
            return None
        
        tree = self.parser.parse(bytes(content, 'utf8'))
        root = tree.root_node
        
        result = {
            'file': rel_path,
            'language': 'jsx' if is_jsx else 'javascript',
            'functions': [],
            'classes': [],
            'imports': [],
            'exports': [],
            'jsx': [],
            'dependencies': [],
            'variables': [],
            'doc': ''
        }
        
        self._traverse(root, result, content)
        result['doc'] = self._generate_summary(result)
        
        return result
    
    def _traverse(self, node, result, content):
        if node.type == 'import_statement':
            self._extract_import(node, result, content)
        elif node.type == 'function_declaration':
            self._extract_function(node, result, content)
        elif node.type == 'arrow_function' or node.type == 'function':
            self._extract_arrow_function(node, result, content)
        elif node.type == 'class_declaration':
            self._extract_class(node, result, content)
        elif node.type == 'export_statement':
            self._extract_export(node, result, content)
        elif node.type == 'jsx_element' or node.type == 'jsx_self_closing_element':
            self._extract_jsx_element(node, result, content)
        elif node.type == 'variable_declaration':
            self._extract_variable(node, result, content)
        
        for child in node.children:
            self._traverse(child, result, content)
    
    def _extract_import(self, node, result, content):
        import_data = {'source': '', 'specifiers': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'string':
                source = content[child.start_byte:child.end_byte].strip('"\'')
                import_data['source'] = source
                if not source.startswith('.') and not source.startswith('/'):
                    result['dependencies'].append(source.split('/')[0])
        result['imports'].append(import_data)
    
    def _extract_function(self, node, result, content):
        func_data = {'name': '', 'params': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'identifier':
                func_data['name'] = content[child.start_byte:child.end_byte]
            elif child.type == 'formal_parameters':
                func_data['params'] = self._extract_params(child, content)
        result['functions'].append(func_data)
    
    def _extract_arrow_function(self, node, result, content):
        parent = node.parent
        func_data = {'name': 'anonymous', 'params': [], 'line': node.start_point[0] + 1}
        
        if parent and parent.type == 'variable_declarator':
            for child in parent.children:
                if child.type == 'identifier':
                    func_data['name'] = content[child.start_byte:child.end_byte]
        
        for child in node.children:
            if child.type == 'formal_parameters':
                func_data['params'] = self._extract_params(child, content)
        
        if func_data['name'] != 'anonymous':
            result['functions'].append(func_data)
    
    def _extract_class(self, node, result, content):
        class_data = {'name': '', 'methods': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'identifier':
                class_data['name'] = content[child.start_byte:child.end_byte]
            elif child.type == 'class_body':
                self._extract_methods(child, class_data, content)
        result['classes'].append(class_data)
    
    def _extract_methods(self, node, class_data, content):
        for child in node.children:
            if child.type == 'method_definition':
                method_data = {'name': '', 'params': []}
                for method_child in child.children:
                    if method_child.type == 'property_identifier':
                        method_data['name'] = content[method_child.start_byte:method_child.end_byte]
                    elif method_child.type == 'formal_parameters':
                        method_data['params'] = self._extract_params(method_child, content)
                if method_data['name']:
                    class_data['methods'].append(method_data)
    
    def _extract_params(self, node, content):
        params = []
        for child in node.children:
            if child.type == 'identifier':
                params.append(content[child.start_byte:child.end_byte])
            elif child.type == 'required_parameter' or child.type == 'optional_parameter':
                for param_child in child.children:
                    if param_child.type == 'identifier':
                        params.append(content[param_child.start_byte:param_child.end_byte])
        return params
    
    def _extract_export(self, node, result, content):
        export_data = {'type': 'unknown', 'name': '', 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'function_declaration' or child.type == 'class_declaration':
                for subchild in child.children:
                    if subchild.type == 'identifier':
                        export_data['name'] = content[subchild.start_byte:subchild.end_byte]
                        export_data['type'] = child.type.replace('_declaration', '')
        result['exports'].append(export_data)
    
    def _extract_variable(self, node, result, content):
        for child in node.children:
            if child.type == 'variable_declarator':
                var_name = ''
                for var_child in child.children:
                    if var_child.type == 'identifier':
                        var_name = content[var_child.start_byte:var_child.end_byte]
                if var_name:
                    result['variables'].append({'name': var_name, 'line': node.start_point[0] + 1})
    
    def _extract_jsx_element(self, node, result, content):
        jsx_data = {'type': '', 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'jsx_opening_element' or child.type == 'jsx_self_closing_element':
                for elem_child in child.children:
                    if elem_child.type == 'identifier':
                        jsx_data['type'] = content[elem_child.start_byte:elem_child.end_byte]
        if jsx_data['type']:
            result['jsx'].append(jsx_data)
    
    def _generate_summary(self, result):
        parts = []
        if result['functions']:
            parts.append(f"{len(result['functions'])} functions")
        if result['classes']:
            parts.append(f"{len(result['classes'])} classes")
        if result['jsx']:
            parts.append(f"{len(result['jsx'])} JSX elements")
        if result['imports']:
            parts.append(f"{len(result['imports'])} imports")
        return f"JavaScript/JSX file with {', '.join(parts) if parts else 'no exports'}"

class TypeScriptParser:
    def __init__(self):
        self.language = Language(tree_sitter_typescript.language_typescript())
        self.parser = Parser(self.language)
        self.tsx_language = Language(tree_sitter_typescript.language_tsx())
        self.tsx_parser = Parser(self.tsx_language)
    
    def parse(self, file_path, rel_path, is_tsx=False):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except:
            return None
        
        parser = self.tsx_parser if is_tsx else self.parser
        tree = parser.parse(bytes(content, 'utf8'))
        root = tree.root_node
        
        result = {
            'file': rel_path,
            'language': 'tsx' if is_tsx else 'typescript',
            'functions': [],
            'classes': [],
            'imports': [],
            'exports': [],
            'jsx': [],
            'dependencies': [],
            'variables': [],
            'doc': ''
        }
        
        self._traverse(root, result, content)
        result['doc'] = self._generate_summary(result)
        
        return result
    
    def _traverse(self, node, result, content):
        if node.type == 'import_statement':
            self._extract_import(node, result, content)
        elif node.type == 'function_declaration':
            self._extract_function(node, result, content)
        elif node.type == 'arrow_function':
            self._extract_arrow_function(node, result, content)
        elif node.type == 'class_declaration':
            self._extract_class(node, result, content)
        elif node.type == 'export_statement':
            self._extract_export(node, result, content)
        elif node.type == 'jsx_element' or node.type == 'jsx_self_closing_element':
            self._extract_jsx_element(node, result, content)
        elif node.type == 'variable_declaration':
            self._extract_variable(node, result, content)
        
        for child in node.children:
            self._traverse(child, result, content)
    
    def _extract_import(self, node, result, content):
        import_data = {'source': '', 'specifiers': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'string':
                source = content[child.start_byte:child.end_byte].strip('"\'')
                import_data['source'] = source
                if not source.startswith('.') and not source.startswith('/'):
                    result['dependencies'].append(source.split('/')[0])
        result['imports'].append(import_data)
    
    def _extract_function(self, node, result, content):
        func_data = {'name': '', 'params': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'identifier':
                func_data['name'] = content[child.start_byte:child.end_byte]
            elif child.type == 'formal_parameters':
                func_data['params'] = self._extract_params(child, content)
        result['functions'].append(func_data)
    
    def _extract_arrow_function(self, node, result, content):
        parent = node.parent
        func_data = {'name': 'anonymous', 'params': [], 'line': node.start_point[0] + 1}
        
        if parent and parent.type == 'variable_declarator':
            for child in parent.children:
                if child.type == 'identifier':
                    func_data['name'] = content[child.start_byte:child.end_byte]
        
        for child in node.children:
            if child.type == 'formal_parameters':
                func_data['params'] = self._extract_params(child, content)
        
        if func_data['name'] != 'anonymous':
            result['functions'].append(func_data)
    
    def _extract_class(self, node, result, content):
        class_data = {'name': '', 'methods': [], 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'type_identifier' or child.type == 'identifier':
                class_data['name'] = content[child.start_byte:child.end_byte]
            elif child.type == 'class_body':
                self._extract_methods(child, class_data, content)
        result['classes'].append(class_data)
    
    def _extract_methods(self, node, class_data, content):
        for child in node.children:
            if child.type == 'method_definition':
                method_data = {'name': '', 'params': []}
                for method_child in child.children:
                    if method_child.type == 'property_identifier':
                        method_data['name'] = content[method_child.start_byte:method_child.end_byte]
                    elif method_child.type == 'formal_parameters':
                        method_data['params'] = self._extract_params(method_child, content)
                if method_data['name']:
                    class_data['methods'].append(method_data)
    
    def _extract_params(self, node, content):
        params = []
        for child in node.children:
            if child.type == 'identifier':
                params.append(content[child.start_byte:child.end_byte])
            elif child.type == 'required_parameter' or child.type == 'optional_parameter':
                for param_child in child.children:
                    if param_child.type == 'identifier':
                        params.append(content[param_child.start_byte:param_child.end_byte])
        return params
    
    def _extract_export(self, node, result, content):
        export_data = {'type': 'unknown', 'name': '', 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'function_declaration' or child.type == 'class_declaration':
                for subchild in child.children:
                    if subchild.type == 'identifier' or subchild.type == 'type_identifier':
                        export_data['name'] = content[subchild.start_byte:subchild.end_byte]
                        export_data['type'] = child.type.replace('_declaration', '')
        result['exports'].append(export_data)
    
    def _extract_variable(self, node, result, content):
        for child in node.children:
            if child.type == 'variable_declarator':
                var_name = ''
                for var_child in child.children:
                    if var_child.type == 'identifier':
                        var_name = content[var_child.start_byte:var_child.end_byte]
                if var_name:
                    result['variables'].append({'name': var_name, 'line': node.start_point[0] + 1})
    
    def _extract_jsx_element(self, node, result, content):
        jsx_data = {'type': '', 'line': node.start_point[0] + 1}
        for child in node.children:
            if child.type == 'jsx_opening_element' or child.type == 'jsx_self_closing_element':
                for elem_child in child.children:
                    if elem_child.type == 'identifier':
                        jsx_data['type'] = content[elem_child.start_byte:elem_child.end_byte]
        if jsx_data['type']:
            result['jsx'].append(jsx_data)
    
    def _generate_summary(self, result):
        parts = []
        if result['functions']:
            parts.append(f"{len(result['functions'])} functions")
        if result['classes']:
            parts.append(f"{len(result['classes'])} classes")
        if result['jsx']:
            parts.append(f"{len(result['jsx'])} JSX elements")
        if result['imports']:
            parts.append(f"{len(result['imports'])} imports")
        return f"TypeScript/TSX file with {', '.join(parts) if parts else 'no exports'}"

class PythonParser:
    def parse(self, file_path, rel_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except:
            return None
        
        try:
            tree = ast.parse(content)
        except:
            return None
        
        result = {
            'file': rel_path,
            'language': 'python',
            'functions': [],
            'classes': [],
            'imports': [],
            'exports': [],
            'jsx': [],
            'dependencies': [],
            'variables': [],
            'doc': ''
        }
        
        self._traverse(tree, result)
        result['doc'] = self._generate_summary(result)
        
        return result
    
    def _traverse(self, node, result):
        if isinstance(node, ast.Import):
            for alias in node.names:
                result['imports'].append({'source': alias.name, 'specifiers': [], 'line': node.lineno})
                result['dependencies'].append(alias.name.split('.')[0])
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                result['imports'].append({'source': node.module, 'specifiers': [a.name for a in node.names], 'line': node.lineno})
                if not node.module.startswith('.'):
                    result['dependencies'].append(node.module.split('.')[0])
        elif isinstance(node, ast.FunctionDef):
            func_data = {
                'name': node.name,
                'params': [arg.arg for arg in node.args.args],
                'line': node.lineno,
                'docstring': ast.get_docstring(node)
            }
            result['functions'].append(func_data)
        elif isinstance(node, ast.ClassDef):
            class_data = {
                'name': node.name,
                'methods': [],
                'line': node.lineno,
                'docstring': ast.get_docstring(node)
            }
            for item in node.body:
                if isinstance(item, ast.FunctionDef):
                    method_data = {
                        'name': item.name,
                        'params': [arg.arg for arg in item.args.args]
                    }
                    class_data['methods'].append(method_data)
            result['classes'].append(class_data)
        elif isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    result['variables'].append({'name': target.id, 'line': node.lineno})
        
        for child in ast.iter_child_nodes(node):
            self._traverse(child, result)
    
    def _generate_summary(self, result):
        parts = []
        if result['functions']:
            parts.append(f"{len(result['functions'])} functions")
        if result['classes']:
            parts.append(f"{len(result['classes'])} classes")
        if result['imports']:
            parts.append(f"{len(result['imports'])} imports")
        return f"Python file with {', '.join(parts) if parts else 'no exports'}"

class DependencyGraph:
    def __init__(self, files_data):
        self.files_data = files_data
    
    def generate(self):
        graph = {
            'nodes': [],
            'edges': [],
            'external_deps': {},
            'file_dependencies': defaultdict(list),
            'dependency_count': {}
        }
        
        for file_data in self.files_data:
            graph['nodes'].append(file_data['file'])
            graph['dependency_count'][file_data['file']] = len(file_data['dependencies'])
            
            for imp in file_data['imports']:
                source = imp['source']
                if source.startswith('.'):
                    resolved = self._resolve_relative(file_data['file'], source)
                    if resolved:
                        graph['edges'].append({
                            'from': file_data['file'],
                            'to': resolved,
                            'line': imp.get('line', 0)
                        })
                        graph['file_dependencies'][file_data['file']].append(resolved)
                else:
                    pkg = source.split('/')[0]
                    if pkg not in graph['external_deps']:
                        graph['external_deps'][pkg] = []
                    graph['external_deps'][pkg].append(file_data['file'])
        
        return graph
    
    def _resolve_relative(self, current_file, import_path):
        current_dir = Path(current_file).parent
        
        for ext in ['.js', '.jsx', '.ts', '.tsx', '.py', '/index.js', '/index.ts', '/index.tsx']:
            test_path = str(current_dir / import_path) + ext
            for file_data in self.files_data:
                if file_data['file'].replace('\\', '/') == test_path.replace('\\', '/'):
                    return file_data['file']
        return None

class CallGraph:
    def __init__(self, files_data):
        self.files_data = files_data
    
    def generate(self):
        graph = {
            'nodes': [],
            'edges': [],
            'function_map': {}
        }
        
        for file_data in self.files_data:
            for func in file_data['functions']:
                func_id = f"{file_data['file']}:{func['name']}"
                graph['function_map'][func['name']] = func_id
                graph['nodes'].append({
                    'id': func_id,
                    'name': func['name'],
                    'file': file_data['file'],
                    'params': func['params'],
                    'line': func['line']
                })
        
        return graph

class DocumentationGenerator:
    def __init__(self, files_data, dependency_data, call_data, output_dir):
        self.files_data = files_data
        self.dependency_data = dependency_data
        self.call_data = call_data
        self.output_dir = Path(output_dir)
        self.docs_dir = self.output_dir / 'docs'
        self.docs_dir.mkdir(exist_ok=True)
    
    def generate_all(self):
        self._generate_summary()
        self._generate_architecture()
        self._generate_files()
        self._generate_components()
        self._generate_dependencies()
        self._generate_routes()
        self._generate_call_graph()
        self._generate_index()
    
    def _generate_summary(self):
        content = "# Repository Summary\n\n"
        content += f"**Total Files Analyzed:** {len(self.files_data)}\n\n"
        
        by_lang = {}
        for file_data in self.files_data:
            lang = file_data['language']
            by_lang[lang] = by_lang.get(lang, 0) + 1
        
        content += "## Files by Language\n\n"
        content += "| Language | Count |\n|----------|-------|\n"
        for lang, count in sorted(by_lang.items(), key=lambda x: -x[1]):
            content += f"| {lang} | {count} |\n"
        
        total_funcs = sum(len(f['functions']) for f in self.files_data)
        total_classes = sum(len(f['classes']) for f in self.files_data)
        total_imports = sum(len(f['imports']) for f in self.files_data)
        
        content += f"\n## Code Statistics\n\n"
        content += f"- **Total Functions:** {total_funcs}\n"
        content += f"- **Total Classes:** {total_classes}\n"
        content += f"- **Total Imports:** {total_imports}\n"
        content += f"- **External Dependencies:** {len(self.dependency_data['external_deps'])}\n"
        content += f"- **Internal Dependencies:** {len(self.dependency_data['edges'])}\n"
        
        content += "\n## Top External Dependencies\n\n"
        sorted_deps = sorted(self.dependency_data['external_deps'].items(), 
                            key=lambda x: len(x[1]), reverse=True)[:10]
        for dep, files in sorted_deps:
            content += f"- **{dep}** (used in {len(files)} files)\n"
        
        with open(self.docs_dir / 'SUMMARY.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_architecture(self):
        content = "# Architecture Overview\n\n"
        
        content += "## Project Structure\n\n"
        dirs = set()
        for file_data in self.files_data:
            parts = Path(file_data['file']).parts
            for i in range(1, len(parts)):
                dirs.add('/'.join(parts[:i]))
        
        for directory in sorted(dirs)[:20]:
            files_count = sum(1 for f in self.files_data if f['file'].startswith(directory))
            content += f"- `{directory}/` ({files_count} files)\n"
        
        content += "\n## Dependency Graph (Top 30 Internal Dependencies)\n\n"
        content += "```mermaid\ngraph TD\n"
        
        for edge in self.dependency_data['edges'][:30]:
            from_node = edge['from'].replace('/', '_').replace('.', '_').replace('-', '_')
            to_node = edge['to'].replace('/', '_').replace('.', '_').replace('-', '_')
            content += f"    {from_node}[{Path(edge['from']).name}] --> {to_node}[{Path(edge['to']).name}]\n"
        
        content += "```\n\n"
        
        content += "## External Dependencies Graph\n\n"
        content += "```mermaid\ngraph LR\n"
        
        for dep, files in list(self.dependency_data['external_deps'].items())[:15]:
            dep_node = dep.replace('-', '_').replace('.', '_').replace('@', '')
            content += f"    {dep_node}[{dep}]\n"
            for file in files[:3]:
                file_node = Path(file).stem.replace('-', '_').replace('.', '_')
                content += f"    {file_node}[{Path(file).name}] --> {dep_node}\n"
        
        content += "```\n\n"
        
        content += "## Most Connected Files\n\n"
        sorted_files = sorted(self.dependency_data['dependency_count'].items(), 
                             key=lambda x: x[1], reverse=True)[:10]
        content += "| File | Dependencies |\n|------|-------------|\n"
        for file, count in sorted_files:
            content += f"| {file} | {count} |\n"
        
        with open(self.docs_dir / 'ARCHITECTURE.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_files(self):
        content = "# Files Documentation\n\n"
        content += f"Total files documented: {len(self.files_data)}\n\n"
        
        for file_data in sorted(self.files_data, key=lambda x: x['file']):
            content += f"## {file_data['file']}\n\n"
            content += f"**Language:** {file_data['language']} | "
            content += f"**Functions:** {len(file_data['functions'])} | "
            content += f"**Classes:** {len(file_data['classes'])} | "
            content += f"**Imports:** {len(file_data['imports'])}\n\n"
            
            content += f"*{file_data['doc']}*\n\n"
            
            if file_data['imports']:
                content += "### Imports\n\n"
                for imp in file_data['imports'][:10]:
                    content += f"- `{imp['source']}`"
                    if 'line' in imp:
                        content += f" (line {imp['line']})"
                    content += "\n"
                content += "\n"
            
            if file_data['functions']:
                content += "### Functions\n\n"
                for func in file_data['functions']:
                    params = ', '.join(func['params']) if func['params'] else ''
                    content += f"#### `{func['name']}({params})`\n\n"
                    content += f"- **Line:** {func['line']}\n"
                    if func.get('docstring'):
                        content += f"- **Documentation:** {func['docstring'][:100]}...\n"
                    content += "\n"
            
            if file_data['classes']:
                content += "### Classes\n\n"
                for cls in file_data['classes']:
                    content += f"#### `class {cls['name']}`\n\n"
                    content += f"- **Line:** {cls['line']}\n"
                    if cls.get('docstring'):
                        content += f"- **Documentation:** {cls['docstring'][:100]}...\n"
                    if cls['methods']:
                        content += "- **Methods:**\n"
                        for method in cls['methods']:
                            params = ', '.join(method['params']) if method['params'] else ''
                            content += f"  - `{method['name']}({params})`\n"
                    content += "\n"
            
            content += "---\n\n"
        
        with open(self.docs_dir / 'FILES.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_components(self):
        content = "# Component Documentation\n\n"
        
        react_files = [f for f in self.files_data if f['language'] in ['jsx', 'tsx']]
        
        if react_files:
            content += f"Total React/TSX files: {len(react_files)}\n\n"
            
            content += "## Component Hierarchy\n\n"
            content += "```mermaid\ngraph TD\n"
            
            for file_data in react_files[:30]:
                if file_data['jsx']:
                    file_node = Path(file_data['file']).stem.replace('-', '_').replace('.', '_')
                    content += f"    {file_node}[{Path(file_data['file']).name}]\n"
                    jsx_types = set(j['type'] for j in file_data['jsx'])
                    for jsx_type in list(jsx_types)[:5]:
                        jsx_node = jsx_type.replace('-', '_')
                        content += f"    {file_node} --> {jsx_node}[{jsx_type}]\n"
            
            content += "```\n\n"
            
            content += "## React Components\n\n"
            for file_data in react_files:
                if file_data['jsx'] or file_data['functions']:
                    content += f"### {file_data['file']}\n\n"
                    
                    if file_data['functions']:
                        component_funcs = [f for f in file_data['functions'] if f['name'][0].isupper()]
                        if component_funcs:
                            content += "**Components:**\n"
                            for func in component_funcs:
                                content += f"- `{func['name']}` (line {func['line']})\n"
                            content += "\n"
                    
                    if file_data['jsx']:
                        jsx_types = {}
                        for j in file_data['jsx']:
                            jsx_types[j['type']] = jsx_types.get(j['type'], 0) + 1
                        content += "**JSX Elements Used:**\n"
                        for jsx_type, count in sorted(jsx_types.items()):
                            content += f"- `<{jsx_type}>` ({count} times)\n"
                        content += "\n"
        else:
            content += "No React/JSX components found.\n"
        
        with open(self.docs_dir / 'COMPONENTS.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_dependencies(self):
        content = "# Dependencies Analysis\n\n"
        
        content += "## External Dependencies\n\n"
        content += f"Total external packages: {len(self.dependency_data['external_deps'])}\n\n"
        
        sorted_deps = sorted(self.dependency_data['external_deps'].items(), 
                            key=lambda x: len(x[1]), reverse=True)
        
        content += "| Package | Files Using | Files |\n|---------|-------------|-------|\n"
        for dep, files in sorted_deps[:30]:
            file_list = ', '.join([Path(f).name for f in files[:3]])
            if len(files) > 3:
                file_list += f" +{len(files)-3} more"
            content += f"| `{dep}` | {len(files)} | {file_list} |\n"
        
        content += "\n## Internal File Dependencies\n\n"
        
        if self.dependency_data['file_dependencies']:
            content += "```mermaid\ngraph TD\n"
            
            for file, deps in list(self.dependency_data['file_dependencies'].items())[:25]:
                file_node = Path(file).stem.replace('-', '_').replace('.', '_')
                for dep in deps[:3]:
                    dep_node = Path(dep).stem.replace('-', '_').replace('.', '_')
                    content += f"    {file_node}[{Path(file).name}] --> {dep_node}[{Path(dep).name}]\n"
            
            content += "```\n\n"
        
        content += "## Dependency Tree (Most Connected)\n\n"
        for file, deps in sorted(self.dependency_data['file_dependencies'].items(), 
                                 key=lambda x: len(x[1]), reverse=True)[:15]:
            content += f"### {file}\n\n"
            content += f"Depends on {len(deps)} files:\n"
            for dep in deps[:10]:
                content += f"- {dep}\n"
            content += "\n"
        
        with open(self.docs_dir / 'DEPENDENCIES.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_routes(self):
        content = "# Routes & Pages\n\n"
        
        route_patterns = ['app/', 'pages/', 'routes/', 'views/']
        route_files = [f for f in self.files_data 
                      if any(pattern in f['file'] for pattern in route_patterns)]
        
        if route_files:
            content += f"Total route files detected: {len(route_files)}\n\n"
            
            content += "## Route Structure\n\n"
            content += "```mermaid\ngraph TD\n"
            
            for file_data in route_files[:30]:
                file_parts = Path(file_data['file']).parts
                if len(file_parts) > 1:
                    parent_node = '_'.join(file_parts[:-1]).replace('-', '_')
                    file_node = file_parts[-1].replace('-', '_').replace('.', '_')
                    content += f"    {parent_node} --> {file_node}[{file_parts[-1]}]\n"
            
            content += "```\n\n"
            
            content += "## Route Files\n\n"
            for file_data in sorted(route_files, key=lambda x: x['file']):
                content += f"### {file_data['file']}\n\n"
                
                if file_data['functions']:
                    content += "**Handlers/Components:**\n"
                    for func in file_data['functions']:
                        content += f"- `{func['name']}` (line {func['line']})\n"
                    content += "\n"
                
                if file_data['exports']:
                    content += "**Exports:**\n"
                    for exp in file_data['exports']:
                        if exp['name']:
                            content += f"- {exp['type']}: `{exp['name']}`\n"
                    content += "\n"
        else:
            content += "No route files detected.\n"
        
        with open(self.docs_dir / 'ROUTES.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_call_graph(self):
        content = "# Call Graph & Functions\n\n"
        
        content += f"Total functions: {len(self.call_data['nodes'])}\n\n"
        
        content += "## Function Distribution by File\n\n"
        func_by_file = defaultdict(list)
        for node in self.call_data['nodes']:
            func_by_file[node['file']].append(node)
        
        sorted_files = sorted(func_by_file.items(), 
                             key=lambda x: len(x[1]), reverse=True)[:20]
        
        content += "| File | Functions |\n|------|----------|\n"
        for file, funcs in sorted_files:
            func_names = ', '.join([f['name'] for f in funcs[:5]])
            if len(funcs) > 5:
                func_names += f" +{len(funcs)-5} more"
            content += f"| {file} | {func_names} |\n"
        
        content += "\n## All Functions\n\n"
        
        for file, funcs in sorted(func_by_file.items()):
            content += f"### {file}\n\n"
            for func in sorted(funcs, key=lambda x: x['line']):
                params = ', '.join(func['params']) if func['params'] else ''
                content += f"- `{func['name']}({params})` - line {func['line']}\n"
            content += "\n"
        
        with open(self.docs_dir / 'CALL_GRAPH.md', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _generate_index(self):
        content = "# Repository Documentation Index\n\n"
        content += "Welcome to the auto-generated documentation.\n\n"
        content += "## Documentation Files\n\n"
        content += "- [📊 Summary](SUMMARY.md) - Overview and statistics\n"
        content += "- [🏗️ Architecture](ARCHITECTURE.md) - Dependency graphs and structure\n"
        content += "- [📁 Files](FILES.md) - Detailed file documentation\n"
        content += "- [⚛️ Components](COMPONENTS.md) - React/JSX components\n"
        content += "- [📦 Dependencies](DEPENDENCIES.md) - External and internal dependencies\n"
        content += "- [🛣️ Routes](ROUTES.md) - Application routes and pages\n"
        content += "- [🔗 Call Graph](CALL_GRAPH.md) - Function listings and relationships\n"
        
        with open(self.docs_dir / 'README.md', 'w', encoding='utf-8') as f:
            f.write(content)

class RepositoryAnalyzer:
    def __init__(self, repo_path, output_dir):
        self.repo_path = Path(repo_path)
        self.output_dir = Path(output_dir)
        self.js_parser = JavaScriptParser()
        self.ts_parser = TypeScriptParser()
        self.py_parser = PythonParser()
        self.files_data = []
    
    def analyze(self):
        print(f"📂 Analyzing repository: {self.repo_path}")
        self._walk_repository()
        
        print(f"\n✅ Parsed {len(self.files_data)} files")
        print("📊 Generating dependency graph...")
        dep_graph = DependencyGraph(self.files_data)
        dependency_data = dep_graph.generate()
        
        print("🔗 Generating call graph...")
        call_graph = CallGraph(self.files_data)
        call_data = call_graph.generate()
        
        print("📝 Generating documentation...")
        doc_gen = DocumentationGenerator(
            self.files_data,
            dependency_data,
            call_data,
            str(self.output_dir)
        )
        doc_gen.generate_all()
        
        return {
            "repository": str(self.repo_path),
            "total_files": len(self.files_data),
            "files": self.files_data,
            "dependencies": dependency_data,
            "call_graph": call_data
        }
    
    def _walk_repository(self):
        for root, dirs, files in os.walk(self.repo_path):
            dirs[:] = [d for d in dirs if not should_skip_directory(d)]
            
            for file in files:
                file_path = Path(root) / file
                language = get_file_language(file)
                
                if language:
                    try:
                        file_data = self._parse_file(file_path, language)
                        if file_data:
                            self.files_data.append(file_data)
                            if len(self.files_data) % 10 == 0:
                                print(f"  Parsed {len(self.files_data)} files...", end='\r')
                    except Exception as e:
                        print(f"⚠️  Error parsing {file_path}: {e}")
    
    def _parse_file(self, file_path, language):
        rel_path = str(file_path.relative_to(self.repo_path)).replace('\\', '/')
        
        if language == 'javascript':
            return self.js_parser.parse(str(file_path), rel_path)
        elif language == 'jsx':
            return self.js_parser.parse(str(file_path), rel_path, is_jsx=True)
        elif language == 'typescript':
            return self.ts_parser.parse(str(file_path), rel_path)
        elif language == 'tsx':
            return self.ts_parser.parse(str(file_path), rel_path, is_tsx=True)
        elif language == 'python':
            return self.py_parser.parse(str(file_path), rel_path)
        
        return None

print("=" * 70)
print("🚀 REPOSITORY ANALYZER - Tree-sitter + AST")
print("=" * 70)
print("\nThis tool analyzes LOCAL repositories and generates:")
print("  📊 Comprehensive statistics")
print("  🏗️ Dependency graphs (Mermaid diagrams)")
print("  📝 Full documentation")
print("  ⚛️ Component analysis")
print("  🔗 Call graphs")
print("=" * 70)

repo_path = input("\n📂 Enter the LOCAL repository path: ").strip()

if repo_path.startswith(('http://', 'https://', 'git@')):
    print("\n❌ ERROR: You provided a URL!")
    print("\n1. Clone first: git clone", repo_path)
    print("2. Run again with local path")
    import sys
    sys.exit(1)

if not os.path.isdir(repo_path):
    print(f"\n❌ ERROR: '{repo_path}' is not a directory!")
    import sys
    sys.exit(1)

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

analyzer = RepositoryAnalyzer(repo_path, str(output_dir))
results = analyzer.analyze()

json_output = output_dir / "analysis.json"
with open(json_output, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\n" + "=" * 70)
print("✅ ANALYSIS COMPLETE!")
print("=" * 70)
print(f"\n📊 Total files analyzed: {results['total_files']}")
print(f"📦 External dependencies: {len(results['dependencies']['external_deps'])}")
print(f"🔗 Internal dependencies: {len(results['dependencies']['edges'])}")
print(f"⚡ Functions found: {len(results['call_graph']['nodes'])}")

print(f"\n📁 Results saved to: {output_dir}/")
print(f"   📄 JSON: analysis.json")
print(f"   📚 Documentation: docs/")
print(f"\n📖 Documentation files generated:")
print(f"   • README.md - Documentation index")
print(f"   • SUMMARY.md - Overview & statistics") 
print(f"   • ARCHITECTURE.md - Dependency graphs")
print(f"   • FILES.md - Detailed file docs")
print(f"   • COMPONENTS.md - React/JSX components")
print(f"   • DEPENDENCIES.md - Dependency analysis")
print(f"   • ROUTES.md - Routes & pages")
print(f"   • CALL_GRAPH.md - Function listings")

print("\n" + "=" * 70)
print("📊 Top 5 External Dependencies:")
print("=" * 70)
sorted_deps = sorted(results['dependencies']['external_deps'].items(), 
                    key=lambda x: len(x[1]), reverse=True)[:5]
for dep, files in sorted_deps:
    print(f"  • {dep}: used in {len(files)} files")

print("\n✨ Open output/docs/README.md to start exploring!")

🚀 REPOSITORY ANALYZER - Tree-sitter + AST

This tool analyzes LOCAL repositories and generates:
  📊 Comprehensive statistics
  🏗️ Dependency graphs (Mermaid diagrams)
  📝 Full documentation
  ⚛️ Component analysis
  🔗 Call graphs
📂 Analyzing repository: E:\AnwitaChakraborty\Projects\Campaign-AI
  Parsed 30 files...
✅ Parsed 31 files
📊 Generating dependency graph...
🔗 Generating call graph...
📝 Generating documentation...

✅ ANALYSIS COMPLETE!

📊 Total files analyzed: 31
📦 External dependencies: 12
🔗 Internal dependencies: 2
⚡ Functions found: 122

📁 Results saved to: output/
   📄 JSON: analysis.json
   📚 Documentation: docs/

📖 Documentation files generated:
   • README.md - Documentation index
   • SUMMARY.md - Overview & statistics
   • ARCHITECTURE.md - Dependency graphs
   • FILES.md - Detailed file docs
   • COMPONENTS.md - React/JSX components
   • DEPENDENCIES.md - Dependency analysis
   • ROUTES.md - Routes & pages
   • CALL_GRAPH.md - Function listings

📊 Top 5 External Depend